In [1]:
from abc import ABC, abstractmethod
from copy import deepcopy
from enum import Enum
import numpy as np
from typing import List
from functools import partial
from operator import itemgetter
from itertools import combinations
from typing import List, Tuple
import random

In [2]:
class Move(Enum):
    '''
    Selects where you want to place the taken piece. The rest of the pieces are shifted
    '''
    TOP = 0
    BOTTOM = 1
    LEFT = 2
    RIGHT = 3


class Player(ABC):
    def __init__(self, genes=None) -> None:
        '''You can change this for your player if you need to handle state/have memory'''


    @abstractmethod
    def make_move(self, game: 'Game') -> tuple[tuple[int, int], Move]:
        '''
        The game accepts coordinates of the type (X, Y). X goes from left to right, while Y goes from top to bottom, as in 2D graphics.
        Thus, the coordinates that this method returns shall be in the (X, Y) format.

        game: the Quixo game. You can use it to override the current game with yours, but everything is evaluated by the main game
        return values: this method shall return a tuple of X,Y positions and a move among TOP, BOTTOM, LEFT and RIGHT
        '''
        pass



In [3]:
class Game(object):
    def __init__(self) -> None:
        self._board = np.ones((5, 5), dtype=np.uint8) * -1
        self.current_player_idx = 1

    def get_board(self) -> np.ndarray:
        '''
        Returns the board
        '''
        return deepcopy(self._board)

    def get_current_player(self) -> int:
        '''
        Returns the current player
        '''
        return deepcopy(self.current_player_idx)

    def print(self):
        '''Prints the board. -1 are neutral pieces, 0 are pieces of player 0, 1 pieces of player 1'''
        print(self._board)

    def check_winner(self) -> int:
        '''Check the winner. Returns the player ID of the winner if any, otherwise returns -1'''
        # for each row
        for x in range(self._board.shape[0]):
            # if a player has completed an entire row
            if self._board[x, 0] != -1 and all(self._board[x, :] == self._board[x, 0]):
                # return the relative id
                return self._board[x, 0]
        # for each column
        for y in range(self._board.shape[1]):
            # if a player has completed an entire column
            if self._board[0, y] != -1 and all(self._board[:, y] == self._board[0, y]):
                # return the relative id
                return self._board[0, y]
        # if a player has completed the principal diagonal
        if self._board[0, 0] != -1 and all(
            [self._board[x, x]
                for x in range(self._board.shape[0])] == self._board[0, 0]
        ):
            # return the relative id
            return self._board[0, 0]
        # if a player has completed the secondary diagonal
        if self._board[0, -1] != -1 and all(
            [self._board[x, -(x + 1)]
             for x in range(self._board.shape[0])] == self._board[0, -1]
        ):
            # return the relative id
            return self._board[0, -1]
        return -1

    def play(self, player1: Player, player2: Player) -> int:
        '''Play the game. Returns the winning player'''
        players = [player1, player2]
        winner = -1
        n_mosse=0
        while winner < 0:
            self.current_player_idx += 1
            self.current_player_idx %= len(players)
            ok = False
            while not ok:
                from_pos, slide = players[self.current_player_idx].make_move(
                    self)
                ok = self.__move(from_pos, slide, self.current_player_idx)
                if ok==True:
                  n_mosse+=1
                  print(f"\n Il giocatore {players[self.current_player_idx]} ha fatto la mossa {n_mosse}")
                  self.print()
            winner = self.check_winner()
        return winner

    def __move(self, from_pos: tuple[int, int], slide: Move, player_id: int) -> bool:
        '''Perform a move'''
        if player_id > 2:
            return False
        # Oh God, Numpy arrays
        prev_value = deepcopy(self._board[(from_pos[1], from_pos[0])])
        acceptable = self.__take((from_pos[1], from_pos[0]), player_id)
        if acceptable:
            acceptable = self.__slide((from_pos[1], from_pos[0]), slide)
            if not acceptable:
                self._board[(from_pos[1], from_pos[0])] = deepcopy(prev_value)
        return acceptable

    def __take(self, from_pos: tuple[int, int], player_id: int) -> bool:
        '''Take piece'''
        # acceptable only if in border
        acceptable: bool = (
            # check if it is in the first row
            (from_pos[0] == 0 and from_pos[1] < 5)
            # check if it is in the last row
            or (from_pos[0] == 4 and from_pos[1] < 5)
            # check if it is in the first column
            or (from_pos[1] == 0 and from_pos[0] < 5)
            # check if it is in the last column
            or (from_pos[1] == 4 and from_pos[0] < 5)
            # and check if the piece can be moved by the current player
        ) and (self._board[from_pos] < 0 or self._board[from_pos] == player_id)
        if acceptable:
            self._board[from_pos] = player_id
        return acceptable

    def __slide(self, from_pos: tuple[int, int], slide: Move) -> bool:
        '''Slide the other pieces'''
        # define the corners
        SIDES = [(0, 0), (0, 4), (4, 0), (4, 4)]
        # if the piece position is not in a corner
        if from_pos not in SIDES:
            # if it is at the TOP, it can be moved down, left or right
            acceptable_top: bool = from_pos[0] == 0 and (
                slide == Move.BOTTOM or slide == Move.LEFT or slide == Move.RIGHT
            )
            # if it is at the BOTTOM, it can be moved up, left or right
            acceptable_bottom: bool = from_pos[0] == 4 and (
                slide == Move.TOP or slide == Move.LEFT or slide == Move.RIGHT
            )
            # if it is on the LEFT, it can be moved up, down or right
            acceptable_left: bool = from_pos[1] == 0 and (
                slide == Move.BOTTOM or slide == Move.TOP or slide == Move.RIGHT
            )
            # if it is on the RIGHT, it can be moved up, down or left
            acceptable_right: bool = from_pos[1] == 4 and (
                slide == Move.BOTTOM or slide == Move.TOP or slide == Move.LEFT
            )
        # if the piece position is in a corner
        else:
            # if it is in the upper left corner, it can be moved to the right and down
            acceptable_top: bool = from_pos == (0, 0) and (
                slide == Move.BOTTOM or slide == Move.RIGHT)
            # if it is in the lower left corner, it can be moved to the right and up
            acceptable_left: bool = from_pos == (4, 0) and (
                slide == Move.TOP or slide == Move.RIGHT)
            # if it is in the upper right corner, it can be moved to the left and down
            acceptable_right: bool = from_pos == (0, 4) and (
                slide == Move.BOTTOM or slide == Move.LEFT)
            # if it is in the lower right corner, it can be moved to the left and up
            acceptable_bottom: bool = from_pos == (4, 4) and (
                slide == Move.TOP or slide == Move.LEFT)
        # check if the move is acceptable
        acceptable: bool = acceptable_top or acceptable_bottom or acceptable_left or acceptable_right
        # if it is
        if acceptable:
            # take the piece
            piece = self._board[from_pos]
            # if the player wants to slide it to the left
            if slide == Move.LEFT:
                # for each column starting from the column of the piece and moving to the left
                for i in range(from_pos[1], 0, -1):
                    # copy the value contained in the same row and the previous column
                    self._board[(from_pos[0], i)] = self._board[(
                        from_pos[0], i - 1)]
                # move the piece to the left
                self._board[(from_pos[0], 0)] = piece
            # if the player wants to slide it to the right
            elif slide == Move.RIGHT:
                # for each column starting from the column of the piece and moving to the right
                for i in range(from_pos[1], self._board.shape[1] - 1, 1):
                    # copy the value contained in the same row and the following column
                    self._board[(from_pos[0], i)] = self._board[(
                        from_pos[0], i + 1)]
                # move the piece to the right
                self._board[(from_pos[0], self._board.shape[1] - 1)] = piece
            # if the player wants to slide it upward
            elif slide == Move.TOP:
                # for each row starting from the row of the piece and going upward
                for i in range(from_pos[0], 0, -1):
                    # copy the value contained in the same column and the previous row
                    self._board[(i, from_pos[1])] = self._board[(
                        i - 1, from_pos[1])]
                # move the piece up
                self._board[(0, from_pos[1])] = piece
            # if the player wants to slide it downward
            elif slide == Move.BOTTOM:
                # for each row starting from the row of the piece and going downward
                for i in range(from_pos[0], self._board.shape[0] - 1, 1):
                    # copy the value contained in the same column and the following row
                    self._board[(i, from_pos[1])] = self._board[(
                        i + 1, from_pos[1])]
                # move the piece down
                self._board[(self._board.shape[0] - 1, from_pos[1])] = piece
        return acceptable

In [4]:
import random
class RandomPlayer(Player):
    def __init__(self) -> None:
        super().__init__()
    def make_move(self, game: 'Game') -> tuple[tuple[int, int], Move]:
        from_pos = (random.randint(0, 4), random.randint(0, 4))
        move = random.choice([Move.TOP, Move.BOTTOM, Move.LEFT, Move.RIGHT])
        return from_pos, move


class MyPlayer(Player):
    def __init__(self) -> None:
        super().__init__()
    def make_move(self, game: 'Game') -> tuple[tuple[int, int], Move]:
        from_pos = (random.randint(0, 4), random.randint(0, 4))
        move = random.choice([Move.TOP, Move.BOTTOM, Move.LEFT, Move.RIGHT])
        return from_pos, move

In [5]:
class MinMaxPlayer(Player):
    def __init__(self, max_depth,id):
        super().__init__()
        self.max_depth = max_depth
        self.id=id

    def make_move(self, game):
        alpha = float('-inf')
        beta = float('inf')
        _, move = self.minimax(game, self.max_depth, alpha, beta, True)
        return self.find_best_move_position(game, move), move

    def minimax(self, game, depth, alpha, beta, maximizing_player):
      gamecopy = deepcopy(game)
      alpha
      if depth == 0 or game.check_winner() != -1:
        return self.evaluate_board(game), None

      legal_moves = self.get_legal_moves(gamecopy)

      if maximizing_player:
        max_eval = float('-inf')
        best_move = None

        for move in legal_moves:
            game_copy = deepcopy(game)
            position = self.find_best_move_position(game, move)
            game_copy._Game__move(position, move, game_copy.current_player_idx)
            eval, _ = self.minimax(game_copy, depth - 1, alpha, beta, False)

            if eval > max_eval:
                max_eval = eval
                best_move = move

            alpha = max(alpha, eval)
            if alpha >= beta:
                break

        return max_eval, best_move

      else:
        min_eval = float('inf')
        best_move = None

        for move in legal_moves:
            game_copy = deepcopy(game)
            position = self.find_best_move_position(game, move)
            game_copy._Game__move(position, move, game_copy.current_player_idx)
            eval, _ = self.minimax(game_copy, depth - 1, alpha, beta, True)

            if eval < min_eval:
                min_eval = eval
                best_move = move

            beta = min(beta, eval)
            if alpha >= beta:
                break

        return min_eval, best_move

    def evaluate_board(self, game):
       board = game.get_board()
       punteggio = 0
       for i in range(5):
        # Righe
           valori_riga = board[i, :]
           punteggio += self.valuta_linea(valori_riga)
        # Colonne
           valori_colonna = board[:, i]
           punteggio += self.valuta_linea(valori_colonna)

    # Controlla le diagonali
       valori_diag1 = np.diag(board)
       valori_diag2 = np.diag(np.fliplr(board))
       punteggio += self.valuta_linea(valori_diag1)
       punteggio += self.valuta_linea(valori_diag2)

    # Valuta la sottomatrice 3x3
       sottomatrice = board[1:4, 1:4]
       numero_di_zero_sottomatrice = np.sum(sottomatrice == 0)
       numero_di_uni_sottomatrice=np.sum(sottomatrice==1)
    # Assegna punteggi in base al numero di zeri nella sottomatrice
       if numero_di_zero_sottomatrice >= 3 and self.id==0:
           punteggio += 300  # Punteggio aggiuntivo se ci sono 3 o più zeri nella sottomatrice
       elif numero_di_uni_sottomatrice>=3 and self.id==1:
           punteggio+=300
    # Strategia per bloccare le mosse dell'avversario
       for i in range(5):
          if np.all(board[i, :] == 1) and self.id==0:  # Controllo righe con potenziali allineamenti dell'avversario
            punteggio += 250000

          if np.all(board[:, i] == 1) and self.id==0:  # Controllo colonne con potenziali allineamenti dell'avversario
            punteggio += 250000

          if np.all(board[i, :] == 0) and self.id==1:  # Controllo righe con potenziali allineamenti dell'avversario
            punteggio += 250000

          if np.all(board[:, i] == 0) and self.id==1:  # Controllo colonne con potenziali allineamenti dell'avversario
            punteggio += 250000
          return punteggio

    def valuta_linea(self, linea):
      conta_0 = np.sum(linea == 0)
      conta_1 = np.sum(linea == 1)
    # Assegna punteggi in base al conteggio
      if self.id==0:
        if conta_0 >= 4:
           punteggio = 100000  # Punteggio elevato per allineamenti di 4 o più zeri
           return punteggio
        elif conta_1 >= 4:
           punteggio = -1000  # Punteggio basso per allineamenti di 4 o più uni
        else:
           punteggio = conta_0 - conta_1


      elif self.id==1:

        if conta_1 >= 4:
           punteggio = 100000  # Punteggio elevato per allineamenti di 4 o più uni
           return punteggio
        elif conta_0 >= 4:
           punteggio = -1000  # Punteggio basso per allineamenti di 4 o più zeri
        else:
           punteggio = conta_0 - conta_1

      return punteggio


    def pezzo_legale(self,from_pos):
              acceptable: bool = (
            # check if it is in the first row
            (from_pos[0] == 0 and from_pos[1] < 5)
            # check if it is in the last row
            or (from_pos[0] == 4 and from_pos[1] < 5)
            # check if it is in the first column
            or (from_pos[1] == 0 and from_pos[0] < 5)
            # check if it is in the last column
            or (from_pos[1] == 4 and from_pos[0] < 5)
            # and check if the piece can be moved by the current player
        )
              if acceptable==True:
                return True
              return False

    def verifica_sottomatrice(self,matrice):
    # Verifica che la matrice sia almeno 5x5
      if matrice.shape != (5, 5):
        raise ValueError("La matrice deve essere 5x5")
    # Estrai la sottomatrice 3x3 iniziando dalla riga 1 e colonna 1
      sottomatrice = matrice[1:4, 1:4]
    # Conta il numero di elementi nella sottomatrice che sono uguali a 1
      numero_di_uno = np.sum(sottomatrice == 1)
      numero_di_zeri=np.sum(sottomatrice==0)
    # Verifica se il numero di 1 è minore di 5
      if self.id==0:
         return numero_di_uno < 3
      return numero_di_zeri < 3

    def get_legal_moves(self,game):
        gamecopy=deepcopy(game)
        lista_mosse_pos=[]
        for _ in range(50): ##Aumento di molto il range
           move=random.choice(list(Move))
           from_pos=(random.randint(0, 4), random.randint(0, 4))
           ok= gamecopy._Game__move(from_pos, move, gamecopy.current_player_idx)
           if self.pezzo_legale(from_pos)==True and ok==True:
               lista_mosse_pos.append(move)
           elif ok==True:
               lista_mosse_pos.append(move)
        return lista_mosse_pos #contiene solo la mossa legale

    def find_best_move_position(self, game,move):
        gamecopy=deepcopy(game)
        position=(0,0)
        for _ in range(50):
           from_pos=(random.randint(0, 4), random.randint(0, 4))
           ok=gamecopy._Game__move(from_pos, move, gamecopy.current_player_idx)
           matrice=gamecopy.get_board()
           if ok==True and gamecopy.check_winner()==self.id:
              position=from_pos
              return position
           elif ok==True and self.verifica_sottomatrice(matrice)==True:
              position=from_pos
           else:
             from_pos1=(random.randint(0, 4), random.randint(0, 4))
             ok1=gamecopy._Game__move(from_pos1, move, gamecopy.current_player_idx)
             if ok1==True:
                position=from_pos1
        return position

In [6]:
    vince0=0
    vince1=0
    i=0
    for _ in range(100):
     i=i+1
     g = Game() ##inizializzo il gioco

     print("Tabella iniziale")
     g.print() ##stampo la tabella iniziale per completezza

     player2 = MinMaxPlayer(3,1) ##inizializzo il giocatore 1 con la strategia MinMax
     player1=RandomPlayer()

     winner = g.play(player1, player2) ##verifichiamo se abbiamo un vincitore

     print(f"Winner: Player {winner}") ##Se abbiamo un vincitore entro un certo range di mosse che ho impostato a 150 totali per evitare che il gioco vada in loop allora lo stampo

     if winner==0:
         vince0+=1
     else:
         vince1+=1
     print(f"Il numero di volte in cui vince il player Random è {vince0} mentre il player MinMax vince {vince1} volte")

Output streaming troncato alle ultime 5000 righe.
 [ 1  1 -1 -1 -1]
 [ 0  1  1  0  0]]

 Il giocatore <__main__.RandomPlayer object at 0x7d6148301c60> ha fatto la mossa 35
[[ 1  0  1 -1  0]
 [ 0  0 -1 -1  1]
 [ 0  1  0 -1  0]
 [ 1  1 -1  0 -1]
 [ 0  1  1  0  0]]

 Il giocatore <__main__.MinMaxPlayer object at 0x7d6148301c00> ha fatto la mossa 36
[[ 1  0  1 -1  0]
 [ 0  0 -1 -1  1]
 [ 0  1  0 -1  0]
 [ 1  1 -1  0 -1]
 [ 1  0  1  0  0]]

 Il giocatore <__main__.RandomPlayer object at 0x7d6148301c60> ha fatto la mossa 37
[[ 0  0  1 -1  0]
 [ 1  0 -1 -1  1]
 [ 0  1  0 -1  0]
 [ 1  1 -1  0 -1]
 [ 1  0  1  0  0]]
Winner: Player 0
Il numero di volte in cui vince il player Random è 16 mentre il player MinMax vince 58 volte
Tabella iniziale
[[-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]]

 Il giocatore <__main__.RandomPlayer object at 0x7d61483024a0> ha fatto la mossa 1
[[ 0 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1]
 [-1 -1 -1 